In [ ]:
import numpy as np
import pandas as pd
import math
from matplotlib import pyplot as plt
import matplotlib as mpl
import seaborn as sns
from datetime import date

In [ ]:
n = 2
A = 85
D = 0
days = 365 

X = np.zeros(n)
X[0] = 1000000
X[1] = 1000000
X1 = np.zeros(days+1)
X2 = np.zeros(days+1)


S = np.zeros(days)
U = np.zeros(days)
Exch_rt = np.zeros(days)
TC = np.zeros(days)
TC_type = np.zeros(days)

import random

s = []
p = 0.5
for i in range(0,days):
    s += random.choices([1, -1], weights=[p, 1-p])
    U[i] = s[i]
    
delta = 0.0003

S[0] = 1
for i in range(1,days):
    S[i] = S[i-1]*math.exp(delta*U[i-1])

D = X[0]+X[1]

In [ ]:
nrows = 1000000 #количество клиентов
    
a = np.random.randint(500, 20000, size=(nrows, 1)) #сколько дает клиент - диапазон
b = np.random.randint(2, size=(nrows, 1)) #какую валюту дает клиент
c = np.zeros(nrows, int) #какую валюту отдаем клиенту 

for i in range(0, nrows): 
    #b[i] = 0
    c[i] = 1 - b[i]
    
df = pd.DataFrame(a, columns = ['amount'])
df['take'] = b
df['give'] = c

In [ ]:
X[0] = 1000000
X[1] = 1000000
X1[0] = X[0]
X2[0] = X[1]
D = X[0]+X[1]
vollume = 0
arb_vollume = 0

In [ ]:
def Exchange(i, j, k, X):
       
        X[i] += k
        
        if i == 0:
            b = (D/(A* n**(n)) - D + X[0])
            d = b**(2)+4*(D**(n+1)/(A*X[0]*n**(2*n)))
            d = d**(1/2)
            X[i] -= k
            return X[1] - (-b + d)/2
        else:
            b = (D/(A* n**(n)) - D + X[1])
            d = b**(2)+4*(D**(n+1)/(A*X[1]*n**(2*n)))
            d = d**(1/2)
            X[i] -= k
            return X[0] - (-b + d)/2

In [ ]:
fee = 0.002
fix_fee = 6
arb_pr = 0
LP_fee1 = 0
LP_fee2 = 0
gas_fee = 0
j = 0

for i in range(0, days):
    give = 1
    Exch_rt[i] = 1 / Exchange(0, 1, give*(1), X)
    #print(i, Exchange(0, 1, give*(1), X), 1 / Exchange(0, 1, give*(1), X))
    for t in range(j, j + 2500):
        dx = Exchange(df['take'].iloc[t], df['give'].iloc[t], df['amount'].iloc[t] * (1 - fee), X)
        LP_fee2 += (df['amount'].iloc[t])*(fee)
        X[df['give'].iloc[t]] -= dx
        X[df['take'].iloc[t]] += df['amount'].iloc[t] * (1-fee)
        vollume += df['amount'].iloc[t]
        if t>0:
            give = 10
            take = Exchange(0, 1, give*(1-fee), X)
            pr_prev = take * S[i] - give
            pr_cur = pr_prev
            h = 100
            while h >= 0.1:
                while pr_cur > 0 and pr_cur >= pr_prev:
                    pr_prev = pr_cur
                    give += h
                    take = Exchange(0, 1, give*(1-fee), X)
                    pr_cur = take * S[i] - give
                give -= h
                pr_cur = pr_prev
                h /= 10
            if pr_prev > fix_fee:
                give -=h
                #print(1, give, take)
                #print(X[0], X[1])
                TC[i] = give
                TC_type[i] = 0
                X[1] -= Exchange(0, 1, give*(1-fee), X)
                X[0] += (give)*(1-fee)
                arb_pr += pr_prev - fix_fee
                gas_fee += fix_fee
                LP_fee1 += (give)*(fee)
                arb_vollume += give
                #print(X[0], X[1])
            #print(pr_prev)
            if pr_prev < 0:
                give = 10
                take = Exchange(1, 0, give*(1-fee), X)
                pr_prev = take / S[i] - give
                pr_cur = pr_prev
                h = 100
                while h >= 0.1:
                    while pr_cur > 0 and pr_cur >= pr_prev:
                        pr_prev = pr_cur
                        give += h
                        take = Exchange(1, 0, give*(1-fee), X)
                        pr_cur = take / S[i] - give
                    give -= h
                    pr_cur = pr_prev
                    h /= 10
                if pr_prev > fix_fee:
                    give -=h
                    #print(0, give, take)
                    TC[i] = give
                    TC_type[i] = 1
                    X[0] -= Exchange(1, 0, give*(1-fee), X)
                    X[1] += (give)*(1-fee)
                    arb_pr += pr_prev - fix_fee
                    gas_fee += fix_fee
                    LP_fee1 += (give)*(fee)
                    arb_vollume += give
    j+=2500
    give = 10
    take = Exchange(0, 1, give*(1-fee), X)
    pr_prev = take * S[i] - give
    pr_cur = pr_prev
    h = 100
    while h >= 0.1:
        while pr_cur > 0 and pr_cur >= pr_prev:
            pr_prev = pr_cur
            give += h
            take = Exchange(0, 1, give*(1-fee), X)
            pr_cur = take * S[i] - give
        give -= h
        pr_cur = pr_prev
        h /= 10
    if pr_prev > fix_fee:
        give -=h
        #print(1, give, take)
        #print(X[0], X[1])
        TC[i] = give
        TC_type[i] = 0
        X[1] -= Exchange(0, 1, give*(1-fee), X)
        X[0] += (give)*(1-fee)
        arb_pr += pr_prev - fix_fee
        gas_fee += fix_fee
        LP_fee1 += (give)*(fee)
        arb_vollume += give
        #print(X[0], X[1])
    #print(pr_prev)
    if pr_prev < 0:
        give = 10
        take = Exchange(1, 0, give*(1-fee), X)
        pr_prev = take / S[i] - give
        pr_cur = pr_prev
        h = 100
        while h >= 0.1:
            while pr_cur > 0 and pr_cur >= pr_prev:
                pr_prev = pr_cur
                give += h
                take = Exchange(1, 0, give*(1-fee), X)
                pr_cur = take / S[i] - give
            give -= h
            pr_cur = pr_prev
            h /= 10
        if pr_prev > fix_fee:
            give -=h
            #print(0, give, take)
            TC[i] = give
            TC_type[i] = 1
            X[0] -= Exchange(1, 0, give*(1-fee), X)
            X[1] += (give)*(1-fee)
            arb_pr += pr_prev - fix_fee
            gas_fee += fix_fee
            LP_fee1 += (give)*(fee)
            arb_vollume += give
    #print(give)
    X1[i+1] = X[0]
    X2[i+1] = X[1]
    #print(X[0], X[1], S[i])

In [ ]:
arb_pr

In [ ]:
gas_fee

In [ ]:
LP_fee1

In [ ]:
LP_fee2

In [ ]:
dpi= 100
fig = plt.figure(dpi = dpi, figsize = (1100/dpi, 700/dpi))
#Exch_rt[0] = 1
plt.plot(range(np.size(Exch_rt)), Exch_rt)
plt.show()

In [ ]:
dpi= 100
fig = plt.figure(dpi = dpi, figsize = (1100/dpi, 700/dpi))
plt.plot(range(np.size(X1)), X1, X2)
plt.show()

In [ ]:
dpi= 100
fig = plt.figure(dpi = dpi, figsize = (1100/dpi, 700/dpi))
plt.plot(range(np.size(S)), S)
plt.show()

In [ ]:
df1 = pd.DataFrame({'': ['Arb_profit', 'gas_fee', 'LP1_profit', 'LP2_profit', 'Vollume', 'Arb_Vollume'], 
                   'Stableswap': ["%.4f"%(arb_pr), "%.4f"%gas_fee, "%.4f"%LP_fee1, "%.4f"%LP_fee2, "%.4f"%vollume, "%.4f"%arb_vollume]
                  })
print(df1)